[View in Colaboratory](https://colab.research.google.com/github/ameasure/try_git/blob/master/Finetune.ipynb)

Welcome to Google Colab!

In [0]:
!pip install tables
import tensorflow
import keras

print('tensorflow:', tensorflow.__version__)
print('keras:', keras.__version__)

tensorflow: 1.11.0-rc1
keras: 2.1.6


In [0]:
print('hello')

hello
